In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [7]:
mnist.train.labels.shape

(55000, 10)

In [4]:
x = tf.placeholder(tf.float32,shape = [2,3])
y = tf.placeholder(tf.float32,shape = [2,3])

mulOp = tf.multiply(x,y)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    val = sess.run([mulOp],feed_dict = {x:[[1,2,3],[4,5,6]], y:[[1,2,3],[7,8,9]]})
    print(val)

[array([[ 1.,  4.,  9.],
       [28., 40., 54.]], dtype=float32)]


In [9]:
import numpy as np
import tensorflow as tf
batch_size = 6
max_query_words = 12
max_passage_words = 50
emb_dim = 100
        
def testDSParser(example_proto):
    features = {"query": tf.FixedLenFeature((max_query_words,emb_dim,1), tf.float32),
              "passage": tf.FixedLenFeature((max_passage_words,emb_dim,1), tf.float32),
              "query_id": tf.FixedLenFeature((1), tf.int64),
              "passage_id": tf.FixedLenFeature((1), tf.int64)}
    parsed_features = tf.parse_single_example(example_proto, features)
    return parsed_features["query"], parsed_features["passage"],parsed_features["query_id"],parsed_features["passage_id"]


dataset = tf.data.TFRecordDataset(filenames = "./trainEmbeddings.tfrecords", compression_type="ZLIB").shuffle(buffer_size = 1000)
dataset = dataset.map(testDSParser)
dataset = dataset.batch(batch_size)
iterator = dataset.make_one_shot_iterator()
query,passage,query_id,passage_id = iterator.get_next()
print(query)
with tf.Session() as sess:
    for i in range(2):
        q,p,qid,pid = sess.run([query,passage,query_id,passage_id])
        print(qid,pid)
    #tmp = q[0].decode("utf-8").split()
    #print(np.array(tmp).reshape(12,50))
    #print(len(q[0].decode("utf-8").split()))

Tensor("IteratorGetNext_3:0", shape=(?, 12, 50, 1), dtype=float32)
[[1135787]
 [1135787]
 [1135787]
 [1135787]
 [1135787]
 [1135787]] [[0]
 [1]
 [2]
 [3]
 [4]
 [5]]
[[1135787]
 [1135787]
 [1135787]
 [1135787]
 [ 281922]
 [ 281922]] [[6]
 [7]
 [8]
 [9]
 [0]
 [1]]


In [ ]:
def parser(fileName):
        record = tf.data.TFRecordDataset(filenames = fileName, compression_type="ZLIB")
        features = {"query": tf.FixedLenFeature((max_query_words,emb_dim,1), tf.float32),
                  "passage": tf.FixedLenFeature((max_passage_words,emb_dim,1), tf.float32),
                  "label": tf.FixedLenFeature((num_classes), tf.int64)}
        parsed_features = tf.parse_single_example(record, features)
        return parsed_features["query"], parsed_features["passage"],parsed_features["label"]
    
    def getDatasetIterator(folderPath,batch_size,SHUFFLE_BATCH_SIZE,mode):
        files = [os.path.join(folderPath, f) for f in os.listdir(folderPath)]
        dataset = tf.data.Dataset.from_tensor_slices(files).shuffle(buffer_size = len(files))
        #dataset = tf.data.TFRecordDataset(filenames = fileName, compression_type="ZLIB").shuffle(buffer_size = len(files))
        #------Follow this order: map -> prefetch -> batch
        dataset = dataset.flat_map(parser)
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(1)
        iterator = dataset.make_initializable_iterator()
        return iterator

#Stratified BAtch

In [87]:
import numpy as np
import tensorflow as tf
import os
batch_size = 2
max_query_words = 12
max_passage_words = 50
emb_dim = 50
num_classes = 2
        
def parseEachRecord(record):
    features = {"query": tf.FixedLenFeature((max_query_words,emb_dim,1), tf.float32),
              "passage": tf.FixedLenFeature((max_passage_words,emb_dim,1), tf.float32),
              "label": tf.FixedLenFeature((num_classes), tf.int64)}
    parsed_features = tf.parse_single_example(record, features)
    return parsed_features["query"], parsed_features["passage"],parsed_features["label"]
    
def parserTextFile(fileName):
    dataset = tf.data.TextLineDataset(fileName).map(lambda line: tf.decode_csv(line,record_defaults = [[1.]]))
    return dataset

def parser(fileName):
    dataset = tf.data.TFRecordDataset(filenames = fileName, compression_type="ZLIB")
    parsed_features = dataset.map(parseEachRecord)
    return parsed_features

def getDatasetIterator(folderPath,batch_size):
    files = [os.path.join(folderPath, f) for f in os.listdir(folderPath)]
    dataset = tf.data.Dataset.from_tensor_slices(files).shuffle(buffer_size = len(files))
    #------Follow this order: map -> prefetch -> batch
    dataset = dataset.flat_map(parser)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)
    iterator = dataset.make_initializable_iterator()
    return iterator

iterator = getDatasetIterator("./TrainData_50",2)
q,p,l = iterator.get_next()

correct_pred = tf.equal(tf.argmax([[0,1],[1,0]],1),tf.argmax(l,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

with tf.Session() as sess:
    sess.run([iterator.initializer])
    for i in range(1):
        tmp = sess.run([l])
        print(tmp)
        #print(index)
        
    #tmp = sess.run([ds])
    #print(type(tmp[0]))
    #q,p,l = sess.run([query,passage,label])
    #print(q.shape)
        

[array([[1, 0],
       [1, 0]], dtype=int64)]


In [10]:
query

<tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=string>

In [37]:
GloveEmbeddings = {}
max_query_words = 12
max_passage_words = 50
emb_dim = 50

def loadEmbeddings(embeddingfile):
    global GloveEmbeddings,emb_dim

    fe = open(embeddingfile,"r",encoding="utf-8",errors="ignore")
    for line in fe:
        tokens= line.strip().split()
        word = tokens[0]
        vec = tokens[1:]
        vec = " ".join(vec)
        GloveEmbeddings[word]=vec
    #Add Zerovec, this will be useful to pad zeros, it is better to experiment with padding any non-zero constant values also.
    GloveEmbeddings["zerovec"] = "0.0 "*emb_dim
    fe.close()
    
loadEmbeddings("./glove.6B.50d.txt")

In [42]:
import gensim
import re
from nltk.corpus import stopwords

f = open("./trainData.tsv","r",encoding="utf-8",errors="ignore")  # Format of the file : query_id \t query \t passage \t label \t passage_id
    
opts = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.ZLIB)
stop_words = set(stopwords.words('english')) 

for line in f:
    tokens = line.strip().lower().split("\t")
    query_id,query,passage,label = int(tokens[0]),tokens[1],tokens[2],int(tokens[3])

    wordsWithoutPunctuation = re.split('\W+', query)
    words = [word for word in wordsWithoutPunctuation if word not in stop_words and len(word) > 0]
    if len(words) == 0:
        words = wordsWithoutPunctuation

    word_count = len(words)
    remaining = max_query_words - word_count  
    if(remaining>0):
        words += ["zerovec"]*remaining # Pad zero vecs if the word count is less than max_query_words
    words = words[:max_query_words] # trim extra words
    print(words)
    #create Query Feature vector 
    query_feature_vector = ""
    for word in words:
        if(word in GloveEmbeddings):
            query_feature_vector += GloveEmbeddings[word]+" "
        else:
            query_feature_vector += GloveEmbeddings["zerovec"]+" "  #Add zerovec for OOV terms
    query_feature_vector = query_feature_vector.strip() 
    print(len(query_feature_vector.split(" ")))
    break
    

['weather', 'hanoi', 'march', 'zerovec', 'zerovec', 'zerovec', 'zerovec', 'zerovec', 'zerovec', 'zerovec', 'zerovec', 'zerovec']
608


In [116]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15131936021751349076
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3215186329
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6959646577381805404
physical_device_desc: "device: 0, name: Quadro M1000M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [8]:
import pandas as pd
data = pd.read_csv("./eval1_unlabelled.tsv",sep = "\t", header = None)

In [11]:
data.shape

(104170, 4)

In [7]:
print(len(data[data[3] == 1]),len(data[data[3] == 0]))
#-------Total count: z
#-------1 count: 524188, 0 Count: 4717692

524188 4717692


In [26]:
import tensorflow as tf
tf.reset_default_graph()

weight = tf.get_variable(name = "weight",shape = (2,2), initializer=tf.initializers.truncated_normal(stddev= 0.1))

y = [[1,0],[0,1]]

loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y,logits=weight))
global_step = tf.train.get_or_create_global_step()

train_step = tf.train.AdamOptimizer(1e-4).minimize(loss = loss,global_step=global_step,name = "tmp")

global_Step = tf.train.get_global_step()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    _,val = sess.run([train_step,global_Step])
    print(sess.run([global_Step])   )
    
    for index in range(5):
        step,val = sess.run([train_step,global_Step])
        #val = sess.run([global_Step])    
        print(val)

[1]
1
2
3
4
5


In [30]:
import tensorflow as tf

y = [[1,2,3],[3,2,1],[7,8,1]]

x = tf.argmax(y,-1)

with tf.Session() as sess:
    print(sess.run([x]))
    print(sess.run([tf.cast(y,tf.float32)]))


[array([2, 0, 1], dtype=int64)]
[array([[1., 2., 3.],
       [3., 2., 1.],
       [7., 8., 1.]], dtype=float32)]


In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 0, 1, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])

import tensorflow as tf
tf
auc, update_op = tf.metrics.auc(y_true, y_scores)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("tf auc: {}".format(sess.run([auc, update_op])))


tf auc: [0.74999976, 0.74999976]


In [ ]:
def getStratifiedDatasetIterator(fileName,batch_size,SHUFFLE_BATCH_SIZE,mode):
    dataset = tf.data.TFRecordDataset(filenames = fileName, compression_type="ZLIB").shuffle(buffer_size = SHUFFLE_BATCH_SIZE)
    #------Follow this order: map -> prefetch -> batch
    dataset = dataset.map(parser)
    ds_pos = dataset.filter(lambda a,b,c: tf.reshape(tf.equal(tf.argmax(c,-1), 1), []))
    ds_neg = dataset.filter(lambda a,b,c: tf.reshape(tf.equal(tf.argmax(c,-1), 0), []))

    dataset = tf.data.Dataset.zip((ds_pos, ds_neg))

    # Each input element will be converted into a two-element `Dataset` using
    # `Dataset.from_tensors()` and `Dataset.concatenate()`, then `Dataset.flat_map()`
    # will flatten the resulting `Dataset`s into a single `Dataset`.
    dataset = dataset.flat_map(
        lambda ex_pos, ex_neg: tf.data.Dataset.from_tensors(ex_pos).concatenate(
            tf.data.Dataset.from_tensors(ex_neg)))

    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(1)
    iterator = dataset.make_initializable_iterator()
    return iterator


In [ ]:
import tensorflow as tf

dataset = tf.dataset.